# Lab 4-1 LSTM example for IMDB dataset

## ref: https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py

Trains an LSTM model on the IMDB sentiment classification task.
The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.

## Notes
- RNNs are tricky. Choice of batch size is important, choice of loss and optimizer is critical, etc. Some configurations won't converge.
- LSTM loss decrease patterns during training can be quite different from what you see with CNNs/MLPs/etc.

In [1]:
from __future__ import print_function

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

In [2]:
# Fix tensorflow GPU allocation

#%% GPU memory fix
import tensorflow as tf
from tensorflow import keras
def get_session(gpu_fraction=0.25):    
	gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction, allow_growth=True)    
	return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
keras.backend.set_session(get_session())


In [3]:
max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)
Build model...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train...
Train on 25000 samples, validate on 25000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/15
25000/25000 [==============================] - 133s 5ms/sample - loss: 0.4682 - acc: 0.7789 - val_loss: 0.3941 - val_acc: 0.8260
Epoch 2/15
25000/25000 [==============================] - 116s 5ms/sample - loss: 0.3031 - acc: 0.8750 - val_loss: 0.3868 - val_acc: 0.8292
Epoch 3/15
25000/25000 [==============================] - 107s 4ms/sample - loss: 0.2212 - acc: 0.9140 - val_loss: 0.4033 - val_acc: 0.8367
Epoch 4/15
25000/25000 [==============================] - 102s 4ms/sample - loss: 0.1530 - acc: 0.9433 - val_loss: 0.5292 - val_